In [5]:
""" Funciones para cada metodo de resolucion de EDOs """

import numpy as np

""" 
    Euler
    @param to: tiempo inicial
    @param h: paso
    @param n: numero de iteraciones a realizar (implicitamente esto define el tf)
    @param yo: condiciones iniciales, su tipo y tamaño en caso de ser un arreglo definen 
                si se trata de una sola ecuacion o un sistema
    @param f: callback a la funcion de carga, que debe tener el siguiente prototipo:
                f(tn, yn) -> devuelve un elemento del mismo tipo y tamaño que yn (escalar o arreglo en caso de un sistema)
"""
def euler(to, h, n, f, yo):
    if (type(yo) is int) or (type(yo) is float):
        yo = [yo]
    y = np.zeros((n+1, len(yo)))
    y[0] = yo
    t = to
    for k in range(1, n+1):
        y[k] = y[k-1] + h*f(t,y[k-1])
        t = t + h
    return y

""" 
    Heun
    @param to: tiempo inicial
    @param h: paso
    @param n: numero de iteraciones a realizar (implicitamente esto define el tf)
    @param yo: condiciones iniciales, su tipo y tamaño en caso de ser un arreglo definen 
                si se trata de una sola ecuacion o un sistema
    @param f: callback a la funcion de carga, que debe tener el siguiente prototipo:
                f(tn, yn) -> devuelve un elemento del mismo tipo y tamaño que yn (escalar o arreglo en caso de un sistema)
"""
def heun(to, h, n, f, yo):
    if (type(yo) is int) or (type(yo) is float):
        yo = [yo]
    y = np.zeros((n+1, len(yo)))
    y[0] = yo
    K1 = 0
    K2 = 0
    t = to
    for k in range(1, n+1):
        K1 = f(t, y[k-1])
        t = t + h
        K2 = f(t, y[k-1] + h*K1)
        y[k] = y[k-1] + h/2*(K1 + K2)
    return y

""" 
    Taylor
    @param order: orden del metodo requerido
    @param to: tiempo inicial
    @param h: paso
    @param n: numero de iteraciones a realizar (implicitamente esto define el tf)
    @param yo: condiciones iniciales, su tipo y tamaño en caso de ser un arreglo definen 
                si se trata de una sola ecuacion o un sistema
    @param yderiv: callback a las derivadas de y. Debe hacer {order} callbacks, que deben tener el siguiente prototipo:
                yderiv[i](tn, yn) -> devuelve un elemento del mismo tipo y tamaño que yn (escalar o arreglo en caso de un sistema)
"""
def taylor(order, to, h, n, yderiv, yo):
    if (type(yo) is int) or (type(yo) is float):
        yo = [yo]
    y = np.zeros((n+1, len(yo)))
    y[0] = yo
    t = to
    for k in range(1, n+1):
        y[k] = y[k-1]
        for i in range(order):
            y[k] += h**(i+1)*(yderiv[i])(t, y[k-1])/np.math.factorial(i+1)
        t = t + h
    return y
    
""" Funcion simple para mostrar el resultado y opcionalmente el error si el parametros 'real' no es None """
def print_result(method, n, res, real):
    print(f'{method} con {n} pasos')
    print(res)
    if real is not None:
        print(f'Error: {np.abs(res[-1][-1] - real):.8f}')
    print(' ')

In [3]:
# Ej 1: Metodo de Euler
# yn+1 = yn + h*K1 
# y' = 3y + 3t
def funcion_carga_ej1y2(tn, yn):
    return 3*yn+3*tn

print('EJERCICIO 1')
sol = np.exp(3*0.6)*4/3 - 0.6 - 1/3
print(f'Solucion real: {sol}')
print(' ')
    
yo = 1
to = 0

h = 0.3
n = 2
y = euler(to, h, n, funcion_carga_ej1y2, yo)
print_result('Euler', n, y[1:], sol)

n = 6
h = 0.1
y = euler(to, h, n, funcion_carga_ej1y2, yo)
print_result('Euler', n, y[1:], sol)

EJERCICIO 1
Solucion real: 7.132863285883928
 
Euler con 2 pasos
[[1.9 ]
 [3.88]]
Error: 3.25286
 
Euler con 6 pasos
[[1.3     ]
 [1.72    ]
 [2.296   ]
 [3.0748  ]
 [4.11724 ]
 [5.502412]]
Error: 1.63045
 


In [4]:
# Ej 2: Metodo de Heun
# yn+1 = yn + h/2[K1 + K2]
# K1 = f(tn, yn)
# K2 = f(tn + h, yn + hK1)

# y' = 3y + 3t
print('EJERCICIO 2')
h = 0.3
n = 2
y = heun(to, h, n, funcion_carga_ej1y2, yo)
print_result('Heun', n, y[1:], sol)

n = 6
h = 0.1
y = heun(to, h, n, funcion_carga_ej1y2, yo)
print_result('Heun', n, y[1:], sol)

EJERCICIO 2
Heun con 2 pasos
[[2.44  ]
 [6.1507]]
Error: 0.98216
 
Heun con 6 pasos
[[1.36      ]
 [1.8787    ]
 [2.6108515 ]
 [3.63009527]
 [5.03547813]
 [6.96021809]]
Error: 0.17265
 


In [3]:
# Ejercicio 3
# y' = -y^2 

print('EJERCICIO 3')
to = 0
yo = 1
h = 0.1
n = 3
y = heun(to, h, n, lambda tn, yn: -yn**2, yo)
print_result('Heun', n, y[1:], None)

EJERCICIO 3
Heun con 3 pasos
[[0.9095    ]
 [0.83396215]
 [0.76997115]]
 


In [10]:
# Ejercicio 4: Taylor de tecer orden
# y' = -cos(t)

print('EJERCICIO 4')
sol = -np.sin(0.3) + 1
print(f'Solucion real: {sol}')
print(' ')
to = 0
yo = 1
h = 0.1
n = 3
order = 3
load_functions = [lambda tn, yn: -np.cos(tn), lambda tn, yn: np.sin(tn), lambda tn, yn: np.cos(tn)]
y = taylor(order, to, h, n, load_functions, yo)
print_result(f'Taylor orden {order}', n, y[1:], sol)

EJERCICIO 4
Solucion real: 0.7044797933386604
 
Taylor orden 3 con 3 pasos
[[0.90016667]
 [0.80133125]
 [0.70448128]]
Error: 0.00000149
 


In [9]:
# Ejercicio 5: Taylor de segundo orden
# y' = -y^2

print('EJERCICIO 5')
to = 0
yo = 1
h = 0.1
n = 3
order = 2
load_functions = [lambda tn, yn: -yn**2, lambda tn, yn: 2*yn**3]
y = taylor(order, to, h, n, load_functions, yo)
print_result(f'Taylor orden {order}', n, y[1:], None)

EJERCICIO 5
Taylor orden 2 con 3 pasos
[[0.91      ]
 [0.83472571]
 [0.7708651 ]]
 


In [15]:
# Ejercicio 6: Taylor de tercer orden
# y' = ye^-t

print('EJERCICIO 6')
to = 0
yo = 1
h = 0.1
n = 3
order = 3
load_functions = [lambda tn, yn: yn*np.exp(-tn), lambda tn, yn: yn*np.exp(-tn)*(np.exp(-tn) - 1), lambda tn, yn: yn*np.exp(-tn)*(np.exp(-2*tn) - 3*np.exp(-3*tn) + 1)]
y = taylor(order, to, h, n, load_functions, yo)
print_result(f'Taylor orden {order}', n, y[1:], None)

EJERCICIO 6
Taylor orden 3 con 3 pasos
[[1.09983333]
 [1.19880989]
 [1.29607447]]
 


In [17]:
# Ejercicio 7: Euler sistema 2 orden
# y'' + y' + y = 0

print('EJERCICIO 7')
yo = [1, 1]
to = 0

h = 0.5
n = 4
y = euler(to, h, n, lambda tn, yn: np.asarray([-yn[0] - yn[1], yn[0]]), yo)
print_result('Euler', n, y[1:], None)

EJERCICIO 7
Euler con 4 pasos
[[ 0.      1.5   ]
 [-0.75    1.5   ]
 [-1.125   1.125 ]
 [-1.125   0.5625]]
 


In [18]:
# Ejercicio 8: Euler sistema 2 orden
# 2y'' - 5y' - 3y = 45e^2t

def funcion_carga_ej8y9(tn, yn):
    return np.asarray([(5*yn[0] + 3*yn[1] + 45*np.exp(2*tn))/2, yn[0]])

print('EJERCICIO 8')

sol = 4*np.exp(-1/2.0) + 7*np.exp(3*1) - 9*np.exp(2)
print(f'Solucion real: {sol}')
print(' ')

yo = [1, 2]  # yo = [x0, y0] porque xk = [x, y]
to = 0
h = 0.1
n = 10
y = euler(to, h, n, funcion_carga_ej8y9, yo)
print_result('Euler', n, y[1:], sol)

EJERCICIO 8
Solucion real: 76.52337621078834
 
Euler con 10 pasos
[[  3.8          2.1       ]
 [  7.81315621   2.48      ]
 [ 13.49505083   3.26131562]
 [ 21.45777818   4.6108207 ]
 [ 32.52131292   6.75659852]
 [ 47.78126504  10.00872981]
 [ 68.69815385  14.78685632]
 [ 97.21492068  21.6566717 ]
 [135.91147456  31.37816377]
 [188.20777456  44.96931123]]
Error: 31.55406499
 


In [19]:
# Ejercicio 9: Heun sistema 2 orden
# 2y'' - 5y' - 3y = 45e^2t

print('EJERCICIO 9')

sol = 4*np.exp(-1/2.0) + 7*np.exp(3*1) - 9*np.exp(2)
print(f'Solucion real: {sol}')
print(' ')

yo = [1, 2]  # yo = [x0, y0] porque xk = [x, y]
to = 0
h = 0.1
n = 10
y = heun(to, h, n, funcion_carga_ej8y9, yo)
print_result('Heun', n, y[1:], sol)

EJERCICIO 9
Solucion real: 76.52337621078834
 
Heun con 10 pasos
[[  4.4065781    2.24      ]
 [  9.45287794   2.88994785]
 [ 16.81783728   4.1429015 ]
 [ 27.43919053   6.27096832]
 [ 42.60821669   9.65528287]
 [ 64.0983836   14.82692858]
 [ 94.33985045  22.52271185]
 [136.65597633  33.76107536]
 [195.5836262   49.94529691]
 [277.30667198  73.00362993]]
Error: 3.51974628
 


In [20]:
# Ejercicio 10: Taylor de segundo orden
# y' = y^2cos(t)

def y_deriv_1_ej10(tn, yn):
    return yn**2*np.cos(tn)

def y_deriv_2_ej10(tn, yn):
    return yn**2*(2*np.cos(tn)**2-np.sin(tn))

print('EJERCICIO 10')

to = 0
yo = 1
h = 0.1
n = 2
order = 2
y = taylor(order, to, h, n, [y_deriv_1_ej10, y_deriv_2_ej10], yo)
print_result(f'Taylor orden {order}', n, y[1:], None)

h = 0.05
n = 4
order = 2
y = taylor(order, to, h, n, [y_deriv_1_ej10, y_deriv_2_ej10], yo)
print_result(f'Taylor orden {order}', n, y[1:], None)

EJERCICIO 10
Taylor orden 2 con 2 pasos
[[1.11      ]
 [1.24417764]]
 
Taylor orden 2 con 4 pasos
[[1.0525    ]
 [1.11051186]
 [1.1747641 ]
 [1.24610814]]
 
